In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score 
from sklearn.metrics import confusion_matrix 
from bubble_plot.bubble_plot import bubble_plot

In [2]:
import pandas_bokeh
from bokeh.io import show, output_notebook
from bokeh.plotting import  figure
import glob
pandas_bokeh.output_notebook()
pd.set_option('plotting.backend','pandas_bokeh')

Loading BokehJS ...

In [3]:
df_AVDATA = pd.read_csv("/Users/navneetgujjar/Downloads/Kaggle Dataset/AviationData.csv")
df_UScodes = pd.read_csv("/Users/navneetgujjar/Downloads/Kaggle Dataset/USState_Codes.csv")

/Users/navneetgujjar/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# DATA EXPLORATION

In [4]:
df_AVDATA.dtypes

Event.Id                   object
Investigation.Type         object
Accident.Number            object
Event.Date                 object
Location                   object
Country                    object
Latitude                   object
Longitude                  object
Airport.Code               object
Airport.Name               object
Injury.Severity            object
Aircraft.damage            object
Aircraft.Category          object
Registration.Number        object
Make                       object
Model                      object
Amateur.Built              object
Number.of.Engines         float64
Engine.Type                object
FAR.Description            object
Schedule                   object
Purpose.of.flight          object
Air.carrier                object
Total.Fatal.Injuries      float64
Total.Serious.Injuries    float64
Total.Minor.Injuries      float64
Total.Uninjured           float64
Weather.Condition          object
Broad.phase.of.flight      object
Report.Status 

In [5]:
df_AVDATA.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,81533.000000,75881.000000,74772.000000,75349.000000,81370.000000
mean,1.147131,0.652772,0.280921,0.360814,5.311847
std,0.447085,5.521338,1.557476,2.257721,27.899156
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


In [6]:
df_AVDATA.shape

(87282, 31)

In [7]:
df_AVDATA.isnull().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54209
Longitude                 54218
Airport.Code              37960
Airport.Name              35327
Injury.Severity             883
Aircraft.damage            3010
Aircraft.Category         56499
Registration.Number        1317
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          5749
Engine.Type                5891
FAR.Description           56824
Schedule                  74952
Purpose.of.flight          5789
Air.carrier               71311
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          3978
Broad.phase.of.flight     25558
Report.Status              5212
Publication.Date          13653
dtype: i

In [8]:
total_missing = df_AVDATA.isnull().sum()
percent_missing_val = total_missing/ len(df_AVDATA)
percent_missing_df = pd.DataFrame({'Missing Percentage': percent_missing_val})
percent_missing_df.sort_values(ascending=False, by= 'Missing Percentage' )
print(percent_missing_df[percent_missing_df['Missing Percentage'] > 0.1])

                        Missing Percentage
Latitude                          0.621079
Longitude                         0.621182
Airport.Code                      0.434912
Airport.Name                      0.404746
Aircraft.Category                 0.647316
FAR.Description                   0.651039
Schedule                          0.858734
Air.carrier                       0.817018
Total.Fatal.Injuries              0.130623
Total.Serious.Injuries            0.143329
Total.Minor.Injuries              0.136718
Broad.phase.of.flight             0.292821
Publication.Date                  0.156424


In [9]:
cols_to_drop = list(percent_missing_df[percent_missing_df['Missing Percentage'] > 0.63].index)
print(cols_to_drop)
df_AVDATA2 = df_AVDATA.drop(columns= cols_to_drop)

['Aircraft.Category', 'FAR.Description', 'Schedule', 'Air.carrier']


In [10]:
df_UScodes.head()

,US_State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


# Data Cleaning

In [11]:
# Data Cleaning
# The following data cleaning steps were taken:

# Columns with too many missing values are dropped.
# Only records for Accidents in United States will be kept.
# Date will be converted to a datetime object and new columns for year, month and weekday are created.
# Records with same values but different spelling in Airport Name, Registration Number and Make are fixed with regex.
# Amateur built gets converted to boolean.
# Location gets split up in City and State.
# Amount of injured gets removed from Injury Severity.

In [12]:
# To check all the value occurences of top 5 repeating in all the columns

selected_cols = ['Investigation.Type','Airport.Code', 'Airport.Name', 'Injury.Severity','Registration.Number',
                'Make', 'Model', 'Amateur.Built', 'Country']

for cols in df_AVDATA2[selected_cols].columns:
    print(df_AVDATA2[cols].value_counts().nlargest(6))
    print('\n---------------\n')

Accident    83536
Incident     3746
Name: Investigation.Type, dtype: int64

---------------

NONE    1486
PVT      468
APA      158
ORD      147
MRI      135
DEN      115
Name: Airport.Code, dtype: int64

---------------

Private             232
PRIVATE             224
Private Airstrip    151
NONE                145
PRIVATE STRIP       111
PRIVATE AIRSTRIP     92
Name: Airport.Name, dtype: int64

---------------

Non-Fatal    66561
Fatal(1)      6167
Fatal         4959
Fatal(2)      3711
Incident      2219
Fatal(3)      1147
Name: Injury.Severity, dtype: int64

---------------

NONE      344
UNREG     118
None       65
UNK        13
USAF        9
N20752      8
Name: Registration.Number, dtype: int64

---------------

Cessna    22227
Piper     12029
CESSNA     4549
Beech      4329
PIPER      2618
Bell       2134
Name: Make, dtype: int64

---------------

152          2350
172          1722
172N         1150
PA-28-140     918
150           823
172M          781
Name: Model, dtype: int64


In [13]:
df_AVDATA2[(df_AVDATA2['Investigation.Type'] == 'Accident') & (df_AVDATA2['Country'] == 'United States')]
df_AVDATA2

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Engine.Type,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Reciprocating,Personal,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Reciprocating,Personal,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Reciprocating,Personal,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Reciprocating,Personal,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,NaN,Personal,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87277,20211223104439,Accident,GAA22WA054,2021-12-21,"Auxerre, OF",United States,475047N,0032948W,NaN,NaN,...,NaN,Unknown,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
87278,20211228104450,Accident,WPR22LA069,2021-12-22,"Auburn, CA",United States,NaN,NaN,NaN,NaN,...,NaN,Positioning,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
87279,20211227104443,Accident,CEN22FA082,2021-12-26,"Hardy, AR",United States,NaN,NaN,NaN,NaN,...,NaN,Personal,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
87280,20211230104454,Accident,ERA22LA097,2021-12-29,"Marathon, FL",United States,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN


In [14]:
#  dropping values which has both longitude and latitude as Nan
df_AVDATA2.dropna(thresh=2, inplace= True, axis = 0, subset=['Longitude', 'Latitude'])

(87282, 27)

In [15]:
# dates are object type
# changing to datetime
df_AVDATA2['Event.Date'] = pd.to_datetime(df_AVDATA2['Event.Date'],errors='ignore')
df_AVDATA2['Publication.Date'] = pd.to_datetime(df_AVDATA2['Publication.Date'],errors='ignore')

In [16]:
# changing airport names to upper case
df_AVDATA2['Airport.Name'] = df_AVDATA2['Airport.Name'].str.upper()
df_AVDATA2['Airport.Name'].value_counts()

PRIVATE                       463
PRIVATE AIRSTRIP              264
NONE                          230
PRIVATE STRIP                 160
MERRILL FIELD                 109
                             ... 
CIRCLE E                        1
GOODALL FLD.                    1
BIJOU BOTTOM STRIP AIRPORT      1
STUTZ FIELD                     1
BAYCITY MUNICIPAL               1
Name: Airport.Name, Length: 21283, dtype: int64

In [17]:
# Dividing Location in Cities and States and changing location to upper case
df_AVDATA2['City'] = df_AVDATA2['Location'].str.split(',').str[0]
df_AVDATA2['State'] = df_AVDATA2['Location'].str.split(',').str[1]
df_AVDATA2 = df_AVDATA2.drop(columns = 'Location')
df_AVDATA2

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,...,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,City,State
0,20001218X45444,Accident,SEA87LA080,1948-10-24,United States,NaN,NaN,NaN,NaN,Fatal(2),...,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaT,MOOSE CREEK,ID
1,20001218X45447,Accident,LAX94LA336,1962-07-19,United States,NaN,NaN,NaN,NaN,Fatal(4),...,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,1996-09-19,BRIDGEPORT,CA
2,20061025X01555,Accident,NYC07LA005,1974-08-30,United States,36.922223,-81.878056,NaN,NaN,Fatal(3),...,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,2007-02-26,Saltville,VA
3,20001218X45448,Accident,LAX96LA321,1977-06-19,United States,NaN,NaN,NaN,NaN,Fatal(2),...,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,2000-12-09,EUREKA,CA
4,20041105X01764,Accident,CHI79FA064,1979-08-02,United States,NaN,NaN,NaN,NaN,Fatal(1),...,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,1980-04-16,Canton,OH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87277,20211223104439,Accident,GAA22WA054,2021-12-21,United States,475047N,0032948W,NaN,NaN,Fatal,...,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT,Auxerre,OF
87278,20211228104450,Accident,WPR22LA069,2021-12-22,United States,NaN,NaN,NaN,NaN,Non-Fatal,...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaT,Auburn,CA
87279,20211227104443,Accident,CEN22FA082,2021-12-26,United States,NaN,NaN,NaN,NaN,Fatal,...,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaT,Hardy,AR
87280,20211230104454,Accident,ERA22LA097,2021-12-29,United States,NaN,NaN,NaN,NaN,Non-Fatal,...,0.0,3.0,0.0,0.0,NaN,NaN,NaN,NaT,Marathon,FL


In [18]:
# to replace NONE by none 
df_AVDATA2['Registration.Number'].replace(to_replace=['(?i)none'],value=['None'],regex=True,inplace=True)
df_AVDATA2['Registration.Number'].value_counts()

None      411
UNREG     118
UNK        13
USAF        9
N20752      8
         ... 
N6613N      1
N4459P      1
N9326G      1
N39812      1
N1690J      1
Name: Registration.Number, Length: 77695, dtype: int64

In [19]:
# to replace columns containing private with Private
# to replace none by None
df_AVDATA2['Airport.Name'].replace(to_replace=['(?i)^.*private.*$'],value=['Private'],regex=True,inplace=True)
df_AVDATA2['Airport.Name'].value_counts()

df_AVDATA2['Airport.Name'].replace(to_replace=['(?i)none'],value=['None'],regex=True,inplace=True)
df_AVDATA2['Airport.Name'].value_counts()

Private                  1200
None                      230
MERRILL FIELD             109
CENTENNIAL                101
VAN NUYS                   95
                         ... 
TICONDEROGA MUNICIPAL       1
CHUATHBALUK                 1
ROSEDALE AIRPORT            1
WILLARD AIRPORT             1
BAYCITY MUNICIPAL           1
Name: Airport.Name, Length: 21062, dtype: int64

# Data Analysis

In [30]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
# setup Lambert Conformal basemap.
# set resolution=None to skip processing of boundary datasets.
m = Basemap(width=12000000,height=9000000,projection='lcc',
            resolution=None,lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
m.bluemarble()
plt.show()

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'